In [1]:
from openbabel import pybel

In [16]:
# Before doing anything, we define the paths to open the files 

smi_1 = "C:/Users/Avalon/Desktop/Pybel/imatinib.smi"   # Query
smi_2 = "C:/Users/Avalon/Desktop/Pybel/kinase.smi"     # Search


In [17]:
# PART 1

# First of all, we define the Lipinksi rules

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# Now we run each line (molecule) and if it obeys Lipinksi rules, it prints it.

for mol in pybel.readfile('smi',smi_2):
    if lipinski(mol) == True:
        print(mol.write('can').rstrip())


CN1CCN(CC1)Cc1ccc(cc1)C(=O)Nc1ccc(c(c1)Nc1nccc(n1)c1cccnc1)C	Imatinib
CCN(CCNC(=O)c1c(C)[nH]c(c1C)/C=C/1\C(=O)Nc2c1cc(F)cc2)CC	Sunitinib


In [27]:
# PART 2



# First of all, we define the Lipinksi rules (This is not needed since it is already stated in the first one, but just in case you want to do it from 0)

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# Now we run each line (molecule) and if it obeys Lipinksi rules, it prints it.
# Also, we calculate the Tanimoto Score against the first molecule of the query file (smi_1)

counter = 0

for mol2 in pybel.readfile('smi',smi_1):
    if counter == 0:
        for mol in pybel.readfile('smi',smi_2):
            if lipinski(mol) == True:
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                print(str(mol.write('can').rstrip()) + "\t" + str(search_f | query_first_fp))
        counter += 1
        

CN1CCN(CC1)Cc1ccc(cc1)C(=O)Nc1ccc(c(c1)Nc1nccc(n1)c1cccnc1)C	Imatinib	1.0
CCN(CCNC(=O)c1c(C)[nH]c(c1C)/C=C/1\C(=O)Nc2c1cc(F)cc2)CC	Sunitinib	0.3389830508474576


In [32]:
# Part 3

# This part is similar to the part 2, but in this case we need to also print the Tanimoto scores

# First of all, we define the Lipinksi rules (This is not needed since it is already stated in the first one and the second one, but just in case you want to do it from 0)

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# In this case, we will also compare the molecules to the 1st query molecule.
# Also, it needs to match Lipinksi rules and should be shown in order of their Tanimoto Score, so:

counter = 0
dict_molecules = {}
list_values_tanimotos = []


for mol2 in pybel.readfile('smi',smi_1):
    if counter == 0:
        for mol in pybel.readfile('smi',smi_2):                  # THIS IS NOT THE SMI SHOWN IN THE PDF. YOU JUST NEED TO REPLACE THIS FOR THE FILE YOU NEED! THE 1k.smi!
            if lipinski(mol) == True:
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                tanimoto_score = (search_f | query_first_fp)
                mol_id = mol.write('can').rstrip()
                list_values_tanimotos.append(tanimoto_score)
                dict_molecules[tanimoto_score] = mol_id
        counter += 1
        
list_values_tanimotos.sort(reverse=True)

counter2 = 1
for value in list_values_tanimotos:
    for key in dict_molecules.keys():
        if value == key and counter2 <= 10:
            print (str(dict_molecules[key]) + "\t" + str(key))
            counter2+=1


CN1CCN(CC1)Cc1ccc(cc1)C(=O)Nc1ccc(c(c1)Nc1nccc(n1)c1cccnc1)C	Imatinib	1.0
CCN(CCNC(=O)c1c(C)[nH]c(c1C)/C=C/1\C(=O)Nc2c1cc(F)cc2)CC	Sunitinib	0.3389830508474576


In [40]:
# Part 4

# This part is similar to the part 3, but we need to get rid of the "counter". We want to compare all queries against all searched molecules

# First of all, we define the Lipinksi rules (This is not needed since it is already stated in the first, second and third part. But just in case you want to do it from 0)

def lipinski(mol):
    desc = mol.calcdesc()
    return desc['HBD'] <= 5 and desc['HBA1'] <= 10 and desc['MW'] <= 500 and desc['logP'] <= 5

# In this case, we will also compare the molecules to the 1st query molecule.
# Also, it needs to match Lipinksi rules and should be shown in order of their Tanimoto Score, so:


dict_molecules = {}
list_values_tanimotos = []
sum_tanimoto = 0
average_tanimoto = 0

for mol2 in pybel.readfile('smi',smi_1):
        for mol in pybel.readfile('smi',smi_2):                
            if lipinski(mol) == True:
                query_first_fp = mol2.calcfp()
                search_f = mol.calcfp()
                tanimoto_score = (search_f | query_first_fp)
                list_values_tanimotos.append(tanimoto_score)
        if list_values_tanimotos != 0:
            for value in list_values_tanimotos:
                sum_tanimoto += value
            average_tanimoto = sum_tanimoto/len(list_values_tanimotos)
            mol_id = mol.write('can').rstrip()
            dict_molecules[average_tanimoto] = mol_id
            average_tanimoto = 0
            sum_tanimoto = 0
            list_values_tanimotos = []

counter2 = 1

for value in sorted(dict_molecules.keys(), reverse=True):
    if counter2 <= 10:
        print(str(dict_molecules[value]) + "\t" + str(value))
        counter2+=1
        



CCN(CCNC(=O)c1c(C)[nH]c(c1C)/C=C/1\C(=O)Nc2c1cc(F)cc2)CC	Sunitinib	0.6694915254237288
